In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.activations import linear, relu, sigmoid

import logging
logging.getLogger("tensorflow").setLevel(logging.ERROR)
tf.autograph.set_verbosity(0)

## Softmax Function

A multiclass neural network generates N outputs. One output is selected as the predicted answer. In the output layer, a vector $\mathbf{z}$ is generated by a linear function which is fed into a softmax function.

The softmax function can be written:
$$a_j = \frac{e^{z_j}}{ \sum_{k=0}^{N-1}{e^{z_k} }} \tag{1}$$

Where $z = \mathbf{w} \cdot \mathbf{x} + b$ and N is the number of feature/categories in the output layer.  

In [2]:
def softmax(z):
    '''
    Softmax Converts Vectors of Values to Probability

    Args:
        z (ndarray (N, )) : Input data, N features
    Returns:
        a (ndarray (N,)) : Softmax of z
    '''
    exp_z = np.exp(z)
    a = exp_z / np.sum(exp_z)

    return a

## Generting Dataset

In [5]:
def load_data():
    X = np.load("X.npy")
    y = np.load("y.npy")
    return X, y

In [6]:
x, y  = load_data()

In [8]:
print ('The shape of X is: ' + str(x.shape))
print ('The shape of y is: ' + str(y.shape))

The shape of X is: (5000, 400)
The shape of y is: (5000, 1)


In [9]:
tf.random.set_seed(1234) # for consistent results
model = Sequential(
    [                       
        tf.keras.Input(shape=(400,)),
        Dense(25, activation='relu', name='L1'),
        Dense(15, activation='relu', name='L2'),
        Dense(10, activation='linear', name='L3')

    ], name = "my_model" 
)

In [10]:
model.summary()

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 L1 (Dense)                  (None, 25)                10025     
                                                                 
 L2 (Dense)                  (None, 15)                390       
                                                                 
 L3 (Dense)                  (None, 10)                160       
                                                                 
Total params: 10575 (41.31 KB)
Trainable params: 10575 (41.31 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [11]:
[layer1, layer2, layer3] = model.layers
W1,b1 = layer1.get_weights()
W2,b2 = layer2.get_weights()
W3,b3 = layer3.get_weights()
print(f"W1 shape = {W1.shape}, b1 shape = {b1.shape}")
print(f"W2 shape = {W2.shape}, b2 shape = {b2.shape}")
print(f"W3 shape = {W3.shape}, b3 shape = {b3.shape}")

W1 shape = (400, 25), b1 shape = (25,)
W2 shape = (25, 15), b2 shape = (15,)
W3 shape = (15, 10), b3 shape = (10,)


In [13]:
model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001)
)

In [14]:
history = model.fit(
    x, y,
    epochs = 40
)

Epoch 1/40
157/157 [==============================] - 4s 6ms/step - loss: 1.4086
Epoch 2/40
157/157 [==============================] - 1s 5ms/step - loss: 0.5308
Epoch 3/40
157/157 [==============================] - 1s 5ms/step - loss: 0.4000
Epoch 4/40
157/157 [==============================] - 1s 6ms/step - loss: 0.3332
Epoch 5/40
157/157 [==============================] - 1s 5ms/step - loss: 0.2955
Epoch 6/40
157/157 [==============================] - 1s 6ms/step - loss: 0.2665
Epoch 7/40
157/157 [==============================] - 1s 4ms/step - loss: 0.2433
Epoch 8/40
157/157 [==============================] - 1s 4ms/step - loss: 0.2220
Epoch 9/40
157/157 [==============================] - 1s 5ms/step - loss: 0.2093
Epoch 10/40
157/157 [==============================] - 1s 4ms/step - loss: 0.1887
Epoch 11/40
157/157 [==============================] - 1s 5ms/step - loss: 0.1799
Epoch 12/40
157/157 [==============================] - 1s 5ms/step - loss: 0.1686
Epoch 13/40
157/157 [====

In [17]:
image_of_two = x[1015]

prediction = model.predict(image_of_two.reshape(1,400))  # prediction

print(f" predicting a Two: \n{prediction}")
print(f" Largest Prediction index: {np.argmax(prediction)}")

1/1 [==============================] - 0s 474ms/step
 predicting a Two: 
[[-10.939243    2.51797     6.054313    2.427497  -11.3835335  -7.2253385
   -6.3267775   4.152692   -6.6699986  -6.2704263]]
 Largest Prediction index: 2


In [18]:
prediction_p = tf.nn.softmax(prediction)

print(f" predicting a Two. Probability vector: \n{prediction_p}")
print(f"Total of predictions: {np.sum(prediction_p):0.3f}")

 predicting a Two. Probability vector: 
[[3.4576676e-08 2.4164453e-02 8.2983410e-01 2.2074198e-02 2.2173309e-08
  1.4181163e-06 3.4829893e-06 1.2391607e-01 2.4711146e-06 3.6848930e-06]]
Total of predictions: 1.000


In [19]:
yhat = np.argmax(prediction_p)

print(f"np.argmax(prediction_p): {yhat}")

np.argmax(prediction_p): 2
